In [1]:
import sys, os
import csv, json
import pandas as pd
import re
from summarizer import Summarizer
import numpy as np


Read dataframe from .csv

In [2]:
df = pd.read_csv('text.csv')
for index, row in df.iterrows():
    # read the comment as strings
    row.comments = eval(row.comments)


Data preprocessing

In [3]:
df = df.drop_duplicates("subid")
df = df.dropna()
df = df.reset_index(drop=True)

In [4]:
for index, row in df.iterrows():
        row['title'] = row['title'].replace("Update:", "")
        row['title'] = row['title'].replace("[UPDATE]", "")
        row['title'] = row['title'].replace("UPDATE:", "")
        re.sub(r".*https\S+", "", row['title'])

In [5]:

for index, row in df.iterrows():
        row['selftext'] = str(row['selftext'])
        row['selftext'] = re.sub(r".*https\S+", "", row['selftext'])
        row['selftext'] = row['selftext'].replace("\n", "")
        
        split_string = row['selftext'].split("\-Edit: ", 1)
        row['selftext'] = split_string[0]
        
        if not row["comments"]:
            df.drop(index, inplace=True)
            continue
            
        for index, comment in row['comments'].items():
            comment['body'] = comment['body'].replace("Update:", "")
            comment['body'] = comment['body'].replace("[UPDATE]", "")
            comment['body'] = comment['body'].replace("UPDATE:", "")
            comment['body'] = comment['body'].replace("\n", "")
            re.sub(r".*https\S+", "", comment['body'])
        

In [6]:
for index, row in df.iterrows():
    for i, comment in row['comments'].items():
        if i is "0" and comment['upvote'] == 1:
            df.drop(index, inplace=True)
            break
        elif i is "0" and comment['body'] == "[deleted]":
            df.drop(index, inplace=True)
            break
df = df.reset_index(drop=True)
df

,subreddit,subid,title,selftext,comments
0,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,< Original post&#x200B;I ultimately decided ...,"{'0': {'comment_id': 'fykksqh', 'upvote': 2292..."
1,relationship_advice,hmzvgt,My (32F) husband (36M) became a robot and I d...,"Hey everybody, here’s the update. I wrote this...","{'0': {'comment_id': 'fx8a379', 'upvote': 1122..."
2,relationship_advice,hxuhig,My (22M) vegan girlfriend (21F) wants me to ge...,Original post:**TL;DR** My gf is a passionate ...,"{'0': {'comment_id': 'fz8n5dz', 'upvote': 1247..."
3,relationship_advice,gxh3g6,My [36M] wife [34F] keeps tying my boots afte...,I've gotten a few requests for update but di...,"{'0': {'comment_id': 'ft1d8by', 'upvote': 9811..."
4,relationship_advice,j1ur51,yesterday I froze during sex and my girlfriend...,I am 23(M) and I've been raped before. Twice. ...,"{'0': {'comment_id': 'g71g5cb', 'upvote': 1188..."
...,...,...,...,...,...
6280,Nanny,bphfm5,Interview tip.,If you happen to be interviewing for a family ...,"{'0': {'comment_id': 'entebwg', 'upvote': 15, ..."
6281,Nanny,bm4w9z,MB drove me a little nuts this morning,She asked me a couple of days ago if I could t...,"{'0': {'comment_id': 'emtqwvk', 'upvote': 30, ..."
6282,Nanny,ar11ey,When your kids are horrible all day long,"Can’t wait to leave this house today, 3M and 8...","{'0': {'comment_id': 'egk4wan', 'upvote': 38, ..."
6283,Nanny,afjk6u,Just watch my one month old for 50 hours a wee...,I'm sure that $1.60 per hour will be more than...,"{'0': {'comment_id': 'edz8hj8', 'upvote': 52, ..."


In [11]:
split_df = df.copy()
for index, row in split_df.iterrows():
    comment_list = []
    for comment in row['comments'].values():
        if type(comment) is dict:
            comment_list.append(comment)
    row['comments'] = comment_list
    
split_df = split_df.explode('comments')

split = pd.DataFrame(split_df['comments'].tolist())
split_df = split_df.reset_index(drop=True)
split_df = pd.concat([split_df.drop('comments', axis=1), split], axis=1)

split_df = split_df.rename(columns={"body": "comment"})
split_df

,subreddit,subid,title,selftext,comment_id,upvote,comment
0,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,< Original post&#x200B;I ultimately decided ...,fykksqh,22920,It's always good when these posts have an endi...
1,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,< Original post&#x200B;I ultimately decided ...,fykpwq3,7889,As the person who has been down like that befo...
2,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,< Original post&#x200B;I ultimately decided ...,fykwxhh,5026,But imagine if you had done the camera thing t...
3,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,< Original post&#x200B;I ultimately decided ...,fykrebn,3649,"pff a happy and dramaless ending? Lady, this i..."
4,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,< Original post&#x200B;I ultimately decided ...,fykmcdz,1843,"They were fucking, OP."
...,...,...,...,...,...,...,...
56815,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpnzzd,9,[deleted]
56816,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpck6k,5,"Omg this is the best, I swear I laugh the hard..."
56817,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpyq04,5,"“God Damnit” 5M. I asked what he said, he repe..."
56818,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebq2uwf,3,Today as I was driving 4M home from preschool ...


In [17]:
split_df.iloc[1]['selftext']

'  < Original post&#x200B;I ultimately decided against getting a secret camera set up because ironically enough I didn\'t want to betray my partner\'s trust. Though part of me wanted to get one to squash any worries of someone living in my walls, as per some comments said!I did though plan to leave work early, which is something I\'ve never done before. My boss allowed me to leave after a half-day.Upon returning home, nothing seemed amiss. I was expecting another car on the drive or parked outside on the street. There was no other car I didn\'t recognise. Quietly letting myself in, I was immediately confused. In the hallway, there was a pair of shoes I didn\'t recognise, and not only that, they looked like men\'s shoes.Standing in the hallway trying to work out what to do; if I should sneak around or make my presence known, before I could decide, my husband walked out of the kitchen with two cups of tea. By my husband\'s face it was obvious he was surprised to see me.Playing along with

Remove bad quality data

In [20]:
# Remove bad quality data


removed_words = ("Update:", "[UPDATE]", "UPDATE:", "\n:", "[deleted]", "<redacted>", "[removed]", "None")

def replace_fun(string, removed_list):
    for w in removed_list:
        if string and w in string:
            string.replace(w, "")

split_df['comment'] = split_df['comment'].apply(lambda c: replace_fun(c, removed_words) if any(map(str(c).__contains__, removed_words)) else c)
split_df['selftext'] = split_df['selftext'].apply(lambda c: replace_fun(c, removed_words) if any(map(str(c).__contains__, removed_words)) else c)
split_df['selftext'] = split_df['selftext'].apply(lambda c: re.sub(r"\s+< Original post&#x\w+;(.*)&#x200B;(.*)", r"\1", str(c)))

    
split_df.replace("", np.nan, inplace=True)
split_df

,subreddit,subid,title,selftext,comment_id,upvote,comment
0,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykksqh,22920,It's always good when these posts have an endi...
1,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykpwq3,7889,As the person who has been down like that befo...
2,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykwxhh,5026,But imagine if you had done the camera thing t...
3,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykrebn,3649,"pff a happy and dramaless ending? Lady, this i..."
4,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykmcdz,1843,"They were fucking, OP."
...,...,...,...,...,...,...,...
56815,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpnzzd,9,None
56816,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpck6k,5,"Omg this is the best, I swear I laugh the hard..."
56817,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpyq04,5,"“God Damnit” 5M. I asked what he said, he repe..."
56818,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebq2uwf,3,Today as I was driving 4M home from preschool ...


In [21]:

split_df = split_df.dropna()
split_df = split_df.reset_index(drop=True)
split_df

,subreddit,subid,title,selftext,comment_id,upvote,comment
0,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykksqh,22920,It's always good when these posts have an endi...
1,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykpwq3,7889,As the person who has been down like that befo...
2,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykwxhh,5026,But imagine if you had done the camera thing t...
3,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykrebn,3649,"pff a happy and dramaless ending? Lady, this i..."
4,relationship_advice,hu3e0a,I (29F) keep finding long hairs in my bathroo...,I ultimately decided against getting a secret ...,fykmcdz,1843,"They were fucking, OP."
...,...,...,...,...,...,...,...
55404,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpqh6d,13,I wrote this one down last spring because it c...
55405,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpck6k,5,"Omg this is the best, I swear I laugh the hard..."
55406,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebpyq04,5,"“God Damnit” 5M. I asked what he said, he repe..."
55407,Nanny,a5u6dv,The sh*t kids say,5M is always saying funny stuff but I had a pa...,ebq2uwf,3,Today as I was driving 4M home from preschool ...


In [22]:
#pd.DataFrame(split_df['selftext']).to_csv("text_selftext.csv", index=False)
#pd.DataFrame(split_df['comment']).to_csv("text_comment.csv", index=False)

split_df.to_csv("text_reddit_v1.csv", index=False)
split_df.to_json("text_reddit_v1.json", orient="index")

# Covert dataframe into .json format (archived)

In [3]:
df1 = pd.read_csv('text_v1.csv')

print(len(df1))
df_for_jer = df1[0:500]
df_for_queenie = df1[500:1000]
df_for_kary = df1[1000:1500]
df_for_wes = df1[1500:2000]


6285


In [15]:
df_for_jer.to_csv("df_for_jer.csv", index=False)

In [16]:
df_for_queenie.to_csv("df_for_queenie.csv", index=False)

In [17]:
df_for_kary.to_csv("df_for_kary.csv", index=False)

In [4]:
df_for_wes.to_csv("df_for_wes.csv", index=False)

In [155]:
df1 = pd.read_csv('text_v1.csv')
temp_df = df1[['selftext', 'comments']]
temp_df = temp_df.rename(columns={"selftext": "history", "comments": "candidates"})

In [170]:
df2 = pd.DataFrame(columns=['personality'])
temp_df

,history,candidates
0,< Original post&#x200B;I ultimately decided ...,"{'0': ""It's always good when these posts have ..."
1,"Hey everybody, here’s the update. I wrote this...","{'0': 'So so happy for you. Best of luck :)', ..."
2,Original post:**TL;DR** My gf is a passionate ...,{'0': 'Mittens is beautiful and lucky to have ...
3,I've gotten a few requests for update but di...,"{'0': ""I'm glad things have worked out well so..."
4,I am 23(M) and I've been raped before. Twice. ...,{'0': 'You could read her this post. If you wa...
...,...,...
6280,If you happen to be interviewing for a family ...,"{'0': ""If there's a tactful way to ask how man..."
6281,She asked me a couple of days ago if I could t...,{'0': 'The worst part about being a part time ...
6282,"Can’t wait to leave this house today, 3M and 8...",{'0': 'I’m happy the MB had your back!! Way to...
6283,I'm sure that $1.60 per hour will be more than...,"{'0': 'Lady, I don\'t care what you ""prefer."" ..."


In [171]:
for index, row in temp_df.iterrows():
    utterances = row.to_dict()
    utterances["candidates"] = eval(utterances["candidates"])
    df2 = df2.append({"personality": {"0": ""}, "utterances": {"0": utterances}}, ignore_index=True)

df2

,personality,utterances
0,{'0': ''},{'0': {'history': ' < Original post&#x200B;I ...
1,{'0': ''},"{'0': {'history': 'Hey everybody, here’s the u..."
2,{'0': ''},{'0': {'history': 'Original post:**TL;DR** My ...
3,{'0': ''},{'0': {'history': ' I've gotten a few request...
4,{'0': ''},{'0': {'history': 'I am 23(M) and I've been ra...
...,...,...
6280,{'0': ''},{'0': {'history': 'If you happen to be intervi...
6281,{'0': ''},{'0': {'history': 'She asked me a couple of da...
6282,{'0': ''},{'0': {'history': 'Can’t wait to leave this ho...
6283,{'0': ''},{'0': {'history': 'I'm sure that $1.60 per hou...


In [172]:
SPLIT_PERCENTAGE = 0.9
df2 = df2.sample(frac=1).reset_index(drop=True)
train = df2[:int(len(df2)*SPLIT_PERCENTAGE)]
valid = df2[int(len(df2)*SPLIT_PERCENTAGE):]

data = {"train": train.to_dict('records'), "valid": valid.to_dict('records')}

with open('text_v2.json', 'w') as fp:
    json.dump(data, fp, indent=1)


,personality,utterances
0,{'0': ''},"{'0': {'history': 'I am a freshman in HS, from..."
1,{'0': ''},{'0': {'history': 'One time my mother was hamm...
2,{'0': ''},{'0': {'history': '...because I can’t. Posting...
3,{'0': ''},{'0': {'history': 'I've always hated reading l...
4,{'0': ''},{'0': {'history': 'My mom (48) and I have neve...
...,...,...
6280,{'0': ''},{'0': {'history': 'Edit: better late than neve...
6281,{'0': ''},{'0': {'history': 'I work at a decently well k...
6282,{'0': ''},{'0': {'history': 'My husband and I have been ...
6283,{'0': ''},{'0': {'history': 'Alright I just wanna say th...
